In [1]:

import numpy 
import matplotlib.pyplot as plt
import pandas
import math
from keras.models import Sequential
from keras.layers import Dense
from keras.layers import LSTM
from sklearn.preprocessing import MinMaxScaler
from sklearn.metrics import mean_squared_error
import pickle


Using TensorFlow backend.


In [2]:
# fix random seed for reproducibility
numpy.random.seed(7)

Total_dataframe = pandas.read_csv('neighbors_2017_combined_train_and_test_with_predictions_fixed_with_zeros.csv', usecols=[4,3,10,11,13])
PUL_list = numpy.unique(Total_dataframe['PU_DO'])
RMSE_list = [0] * len(PUL_list)
MAPE_list = [0] * len(PUL_list)

#delete warning flag
pandas.options.mode.chained_assignment = None  # default='warn'

for i in range(20):    
    print(i/len(PUL_list))
    PULoc = PUL_list[i]
    
    #Training/testing Split
    dataframe = Total_dataframe
    dataframe = dataframe[dataframe['PU_DO'] == PULoc]
    dataframe = dataframe['time_delta']
    dataset = dataframe.values
    dataset = dataset.astype('float32')
    dataset = numpy.reshape(dataset, (len(dataset),1))


    #Scalar transform
    scaler = MinMaxScaler(feature_range=(0, 1))
    dataset = scaler.fit_transform(dataset)


    dataframe = Total_dataframe
    dataframe = dataframe[dataframe['PU_DO'] == PULoc]
    dataframe['time_delta'] = dataset
    dataframe = dataframe[dataframe['month'] <= 10]
    dataframe2 = dataframe[dataframe['month'] >= 10]
    dataframe['time_delta']
    dataframe2 = dataframe2['time_delta']
    dataset2 = dataframe2.values
    dataset2 = dataset2.astype('float32')
    dataset2 = numpy.reshape(dataset2, (len(dataset2),1))
    

    

    #Training/Testing Data (Train on Jan-Oct, Ttest on Nov-Dec) 370472 is index
    train_size = int(len(dataset) * 0.9)
    test_size = len(dataset) - train_size
    test2_size = len(dataset2)
    train = dataset[0:train_size,:]
    train, test = dataset[0:train_size,:], dataset[train_size:len(dataset),:]
    test2 = dataset2

    def create_dataset(dataset, look_back=1):
        dataX, dataY = [], []
        for i in range(len(dataset)-look_back-1):
            a = dataset[i:(i+look_back), 0]
            dataX.append(a)
            dataY.append(dataset[i + look_back, 0])
        return numpy.array(dataX), numpy.array(dataY)

    # reshape into X=t and Y=t+1
    look_back = 1
    trainX, trainY = create_dataset(train, look_back)
    testX, testY = create_dataset(test, look_back)
    test2X, test2Y = create_dataset(test2, look_back)

    # reshape input to be [samples, time steps, features]
    trainX = numpy.reshape(trainX, (trainX.shape[0], 1, trainX.shape[1]))
    testX = numpy.reshape(testX, (testX.shape[0], 1, testX.shape[1]))
    test2X = numpy.reshape(test2X, (test2X.shape[0], 1, test2X.shape[1]))



    # create and fit the LSTM network
    model = Sequential()
    #model.add(LSTM(1, input_shape=(1, look_back)))
    model.add(LSTM(4))
    model.add(Dense(1))
    model.compile(loss='mean_squared_error', optimizer='adam')
    model.fit(trainX, trainY, epochs=4, batch_size=10, verbose=0)




    # make predictions
    trainPredict = model.predict(trainX)
    testPredict = model.predict(testX)
    test2Predict = model.predict(test2X)
     #invert predictions
    trainPredict = scaler.inverse_transform(trainPredict)
    trainY = scaler.inverse_transform([trainY])
    testPredict = scaler.inverse_transform(testPredict)
    testY = scaler.inverse_transform([testY])
    test2Predict = scaler.inverse_transform(test2Predict)
    test2Y = scaler.inverse_transform([test2Y])
    # calculate root mean squared error
    trainScore = math.sqrt(mean_squared_error(trainY[0], trainPredict[:,0]))
    print('Train Score: %.2f RMSE' % (trainScore))
    testScore = math.sqrt(mean_squared_error(testY[0], testPredict[:,0]))
    print('Test Score: %.2f RMSE' % (testScore))
    test2Score = math.sqrt(mean_squared_error(test2Y[0], test2Predict[:,0]))
    print('Test2 Score: %.2f RMSE' % (test2Score))

           #
    #print('training actual:', trainY[0])
    #print('training predict:', trainPredict[:,0]  )
    #print(numpy.mean(abs(trainY[0]   -trainPredict[:,0]  )))

    #print('testing actual:', testY[0] )
    #print('testing predict:', testPredict[:,0] )
    #print(numpy.mean( abs(testY[0] - testPredict[:,0]  )))

    '''
    # shift train predictions for plotting
    trainPredictPlot = numpy.empty_like(dataset)
    trainPredictPlot[:, :] = numpy.nan
    trainPredictPlot[look_back:len(trainPredict)+look_back, :] = trainPredict
    # shift test predictions for plotting
    testPredictPlot = numpy.empty_like(dataset)
    testPredictPlot[:, :] = numpy.nan
    testPredictPlot[len(trainPredict)+(look_back*2)+1:len(dataset)-1, :] = testPredict
    # plot baseline and predictions
    from matplotlib.pyplot import figure
    figure(num=None, figsize=(8, 6), dpi=80, facecolor='w', edgecolor='k')
    plt.plot(scaler.inverse_transform(dataset))
    plt.plot(trainPredictPlot)
    plt.plot(testPredictPlot)
    plt.title('Trip Duration From Zone 4 to 164 and its LSTM Prediction')
    plt.xlabel('Time Step')
    plt.ylabel('Time')
    '''

    RMSE_list[i] = test2Score
    Test2_MAPE = numpy.mean(abs(test2Y[0]- test2Predict[:,0] ))  
    MAPE_list[i] = Test2_MAPE
    print('Test_MAD:', Test2_MAPE)
    
    #print('Testy:', (testY[0])[0:10])
    #print('Testpredict:', testPredict[:,0])
    
    Test_MAPE = numpy.mean(abs(testY[0]- testPredict[:,0] )  )  
    print('Validation_MAD:', Test_MAPE)

    Test2_MAD = numpy.mean(abs(test2Y[0]- test2Predict[:,0]))
    Test_MAD = numpy.mean(abs(testY[0]- testPredict[:,0]))
    #print('Test_MAD:', Test2_MAPE)
    #print('Validation_MAD:', Test_MAPE)
    
    #Save the Model
    filename = 'Time_LSTM' + str(i)  +'.sav'
    #pickle.dump(model, open(filename, 'wb'))
    

0.0
Train Score: 150.86 RMSE
Test Score: 152.50 RMSE
Test2 Score: 164.09 RMSE
Test_MAD: 120.5514900016044
Validation_MAD: 116.7233670790723
0.0043859649122807015
Train Score: 124.63 RMSE
Test Score: 143.02 RMSE
Test2 Score: 139.29 RMSE
Test_MAD: 102.00474068676257
Validation_MAD: 103.48357524268069
0.008771929824561403
Train Score: 117.02 RMSE
Test Score: 138.57 RMSE
Test2 Score: 126.34 RMSE
Test_MAD: 92.9129186260428
Validation_MAD: 103.98179891963399
0.013157894736842105
Train Score: 121.87 RMSE
Test Score: 153.19 RMSE
Test2 Score: 133.12 RMSE
Test_MAD: 96.95219532921944
Validation_MAD: 108.18277864979781
0.017543859649122806
Train Score: 143.08 RMSE
Test Score: 144.46 RMSE
Test2 Score: 147.52 RMSE
Test_MAD: 109.2277597999678
Validation_MAD: 107.75414079520134
0.021929824561403508
Train Score: 75.48 RMSE
Test Score: 69.05 RMSE
Test2 Score: 70.53 RMSE
Test_MAD: 52.51143402774934
Validation_MAD: 50.807956654198215
0.02631578947368421
Train Score: 91.02 RMSE
Test Score: 83.29 RMSE
Test2

In [6]:
print(sum(RMSE_list)/20)
print(sum(MAPE_list)/20)

126.03489993752494
91.91301377976171


In [ ]:
import numpy
numpy.savetxt("LSTM_Time_MAD.csv", MAD_list, delimiter=",")
numpy.savetxt("LSTM_Time_RMSE.csv", RMSE_list, delimiter=",")

In [14]:

print(filename)

Time_LSTM99.sav


In [3]:
219:range(len([0]))

SyntaxError: illegal target for annotation (<ipython-input-3-b471e228d8a5>, line 1)